# Run models (merged with LoRA adapters) on testsets

In [2]:
torch.cuda.empty_cache() #empty the GPU

NameError: name 'torch' is not defined

## Load and merge adapters with the base LLM

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -----------------------------
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# -----------------------------
# Base model and LoRA adapter paths
base_model_name = "google/gemma-2-9b-it"
adapter_paths = {
    "simplifier": "output_jupyter/simplifier_syn_final_seed42_20251118_195853",
     "highlighter": "output_jupyter/highlighter_syn_final_seed42_20251118_200812",
     "end2end": "output_jupyter/end2end_syn_final_seed42_20251118_201751"
}

# -----------------------------
# Load models and tokenizers
models = {}
tokenizers = {}

for name, path in adapter_paths.items():
    print(f"\nLoading {name} adapter...")

    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizers[name] = tokenizer

    # Load base model in FP16
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        device_map="auto",            # Automatically place layers on GPU
        torch_dtype=torch.float16     # Use FP16 for H100
    )

    # Attach LoRA adapter
    model = PeftModel.from_pretrained(base_model, path)
    model.eval()
    models[name] = model

print("\n✅ All adapters loaded successfully!")


/home/cracle/data/conda/envs/training_gemma/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda

Loading simplifier adapter...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.02s/it]



Loading highlighter adapter...


Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.22s/it]



Loading end2end adapter...


Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.81s/it]
Some parameters are on the meta device because they were offloaded to the cpu.



✅ All adapters loaded successfully!


In [ ]:
from peft import LoraConfig, get_peft_model, PeftModel
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
peft_model = get_peft_model(base_model, lora_config)
trainable = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
print("Trainable LoRA params:", trainable)

Trainable LoRA params: 4472832


/home/cracle/data/conda/envs/training_gemma/lib/python3.10/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/cracle/data/conda/envs/training_gemma/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


## Run some tests to see if everything works

In [ ]:
import torch

# run on one example
def generate_simplified_text(model, tokenizer, instruction, input_text, 
                             max_new_tokens=512, temperature=0.7, top_p=0.9):
    """
    Generates simplified text using the LoRA-adapted Gemma-2-9B model.
    
    Args:
        model: PeftModel with LoRA adapter loaded
        tokenizer: corresponding tokenizer
        instruction: string, the instruction for simplification
        input_text: string, the original German text
        max_new_tokens: int, maximum tokens to generate
        temperature: float, controls randomness (0-1)
        top_p: float, nucleus sampling

    Returns:
        Simplified text as string
    """
    # Build prompt matching training format
    prompt = f"Instruction: {instruction}\nInput: {input_text}\nOutput:"

    # Tokenize prompt
    model_device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model_device) for k, v in inputs.items()}

    # Generate output
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=False,#True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2  # discourage repetition
        )

    # Decode and remove prompt
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # Strip the original prompt from the output
    simplified_text = generated_text[len(prompt):].strip()

    return simplified_text

simplified = generate_simplified_text(
    model=models["simplifier"],
    tokenizer=tokenizers["simplifier"],
    instruction="Instruction: Vereinfache und schreibe den folgenden Text neu in leichter Sprache, ohne neue Informationen hinzuzufügen. Verwende kurze Sätze und einfache Wörter. Alle Informationen sollen erhalten bleiben, auch wenn es Fehler oder unvollständige Teile gibt.",
    input_text="Texte verständlich machen\nInformationen für Alle\n\nUm am gesellschaftlichen Leben teilhaben zu können, müssen Informationen für alle nutzbar sein. Viele Texte sind schwer verständlich und für TEST Menschen mit Lernschwierigkeiten oder Menschen, die die deutsche Sprache erlernen wollen, nicht geeignet. Das ZFI verfügt über geschulte Fachkräfte und übersetzt Texte in Leichte Sprache.\n\nTexte in Leichter Sprache sind für viele Menschen besser zu verstehen.\nWir übersetzen z. B.:\n\n    Informationsbroschüren/Flyer\n    Erklärungen zu Formularen/Gesetzestexte\n    Arbeitsverträge/Mietverträge\n    Leitbilder/Hausordnungen\n    Internetseiten\n\nJeder Text wird von der Zielgruppe auf Verständlichkeit und Lesbarkeit geprüft. Alle Prüfer werden durch das ZFI geschult. Für interessierte Einrichtungen bieten wir Schulungen und Vorträge zu/und in Leichter Sprache an.\n\nInfoflyer zum Büro für Leichte Sprache als pdf.\n",
    #input_text="Viele Schulen in Deutschland wollen ab nächstem Jahr mehr digitale Geräte verwenden, um den Unterricht moderner zu machen und die Schüler besser auf die Zukunft vorzubereiten.",
    max_new_tokens=512
)

print(simplified)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Einfacher Text zur Übersetzung ins Deutsche
Viele Leute haben Probleme damit, Dinge im Internet zu lesen. Es ist sehr schwierig! Wir helfen ihnen dabei, alles zu verstehen. Unsere Mitarbeiterinnen und Mitarbeiter sprechen Deutsch fließend und wissen genau, wie man Sachen einfach erklärt. Sie arbeiten daran, dass jeder etwas versteht – egal ob groß oder klein, jung oder alt. 
Es macht uns Spaß, andere Sprachen zu lernen und sie anderen beizubringen. Wenn du mehr darüber erfahren möchtest, was wir tun, kannst du gerne einen Flyer herunterladen. Du findest ihn hier unten. Dort steht alles drin, was du wissen musst. Und falls du Fragen hast, dann melde dich bei uns. Wir freuen uns darauf, dir weiterzuhelfen!


\nHier ein paar Beispiele dafür, was wir schon gemacht haben:\n- Broschüren / Flugblätter (für verschiedene Themen)\n- Erklärungen zu Formularen / Gesetzen\n- Arbeitsverträge / Mietverträge\n- Leitlinien / Hausordnung\n- Webseiten.\n Jeder Text wurde von einer Gruppe überprüft, um si

## Run each merged model on their testsets

In [ ]:
# run on testsets

import json

def run_inference_on_jsonl(
        input_jsonl_path,
        output_jsonl_path,
        model,
        tokenizer,
        instruction,
        input_field,
        output_field,
        max_new_tokens=2048,
        temperature=0.7,
        top_p=0.9
    ):
    """
    Runs the generate_simplified_text function on each line in a JSONL file
    and writes the results to a new JSONL file.

    Args:
        input_jsonl_path: path to the input JSONL test set
        output_jsonl_path: where to save the output JSONL
        model, tokenizer: your model + tokenizer
        instruction: instruction string for generate_simplified_text
        text_field: field name in each JSONL line containing the input text
        output_field: name of the field to store the model output
        max_new_tokens, temperature, top_p: generation parameters
    """

    count = 0
    with open(input_jsonl_path, "r", encoding="utf-8") as infile, \
         open(output_jsonl_path, "w", encoding="utf-8") as outfile:

        for line in infile:
            if not line.strip():
                continue

            item = json.loads(line)

            # Extract the input text
            input_text = item[input_field]
            instruction_text = item[instruction]

            # Generate prediction
            prediction = generate_simplified_text(
                model=model,
                tokenizer=tokenizer,
                instruction=instruction,
                input_text=input_text,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p
            )

            # Add prediction to output item
            item[output_field] = prediction

            # Write updated JSON line
            outfile.write(json.dumps(item, ensure_ascii=False) + "\n")

            count += 1
            if count % 50 == 0:
                print(f"{count} lines processed...")

    print(f"Done! Saved {count} lines to: {output_jsonl_path}")


In [ ]:
import json
from datasets import load_dataset
from tqdm import tqdm

# -----------------------------
# 1️⃣ Load your test datasets
test_files = {
    "simplifier": "data/Accessibility_Seminar/datasets_for_models_synthetic/simplifier_test_final.jsonl",
    "highlighter": "data/Accessibility_Seminar/datasets_for_models_synthetic/highlighter_test_final.jsonl",
    "end2end": "data/Accessibility_Seminar/datasets_for_models_synthetic/end2end_test_final.jsonl"
}

#datasets = {name: load_dataset("json", data_files=file)["train"] for name, file in test_files.items()}

#run_inference_on_jsonl(test_files["simplifier"])
print("✅ Test datasets loaded!\n")



✅ Test datasets loaded!



In [ ]:
#simplifier

run_inference_on_jsonl(
    input_jsonl_path=test_files["simplifier"],
    output_jsonl_path="data/Accessibility_Seminar/datasets_for_models_synthetic/simplifier_testset_output_final.jsonl",
    model=models["simplifier"],
    tokenizer=tokenizers["simplifier"],
    instruction="instruction",
    input_field="input",       # adjust if your field name is different
    output_field="model_output"
)

Done! Saved 27 lines to: data/Accessibility_Seminar/datasets_for_models_synthetic/simplifier_testset_output_final.jsonl


In [ ]:
# highlighter


run_inference_on_jsonl(
    input_jsonl_path=test_files["simplifier"],
    output_jsonl_path="data/Accessibility_Seminar/datasets_for_models_synthetic/highlighter_testset_output_final.jsonl",
    model=models["highlighter"],
    tokenizer=tokenizers["highlighter"],
    instruction="instruction",
    input_field="output",       # adjust if your field name is different
    output_field="model_output"
)

Done! Saved 27 lines to: data/Accessibility_Seminar/datasets_for_models_synthetic/highlighter_testset_output_final.jsonl


In [ ]:
#end2end
run_inference_on_jsonl(
    input_jsonl_path=test_files["simplifier"],
    output_jsonl_path="data/Accessibility_Seminar/datasets_for_models_synthetic/end2end_testset_output_final.jsonl",
    model=models["end2end"],
    tokenizer=tokenizers["end2end"],
    instruction="instruction",
    input_field="input",       # adjust if your field name is different
    output_field="model_output"
)

Done! Saved 27 lines to: data/Accessibility_Seminar/datasets_for_models_synthetic/end2end_testset_output_final.jsonl
